In [ ]:
import nest_asyncio

nest_asyncio.apply()

# Generate synthetic dataset of questions + doc ids


In [ ]:
# DEFAULT_QA_GENERATE_PROMPT_TMPL = """\
# Context information is below.

# ---------------------
# {context_str}
# ---------------------

# Given the context information and not prior knowledge.
# generate only questions based on the below query.

# You are a Teacher/ Professor with expertise in the field of AI. Your task is to setup \
# {num_questions_per_chunk} questions for an upcoming \
# quiz/examination. The questions should be diverse in nature \
# across the document. Restrict the questions to the \
# context information provided."
# """

STUDENT_AI_QUESTIONS_GENERATE_PROMPT_TMPL = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information above, generate {num_questions_per_chunk} questions that a student might ask about AI, specifically related to the information provided in the context, 
but without the questions mentioning the context information.

You are simulating curious students who are learning about AI. Your task is to create questions that:
1. Reflect genuine curiosity about the topics covered in the context.
2. Vary in complexity, from basic clarifications to more advanced inquiries.
3. Demonstrate a student's desire to understand AI concepts better.
4. Can be answered using the information provided in the context.

The questions should be diverse and cover different aspects of the content. Do not use any prior knowledge beyond what's given in the context. Ensure that each question you generate can be answered using the information provided in the context.
"""

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.llms.gemini import Gemini

# llm = Gemini(model="models/gemini-1.5-flash-latest", temperature=1, max_tokens=1000)
llm = OpenAI(
    api_key="",
    model="gpt-4o-mini",
    temperature=1,
    max_tokens=1000,
)

In [ ]:
import json
import re
import uuid
import warnings
import asyncio
from typing import Dict, List, Tuple

from llama_index.core.llms.llm import LLM
from llama_index.core.schema import MetadataMode, TextNode
from tqdm.asyncio import tqdm as async_tqdm
from llama_index.core.llama_dataset.legacy.embedding import EmbeddingQAFinetuneDataset
from llama_index.core import Document


async def generate_qa_embedding_pairs(
    nodes: List[TextNode],
    llm: LLM,
    qa_generate_prompt_tmpl: str = STUDENT_AI_QUESTIONS_GENERATE_PROMPT_TMPL,
    num_questions_per_chunk: int = 1,
    max_concurrent: int = 10,
    delay: float = 0.5,
) -> EmbeddingQAFinetuneDataset:
    """Generate examples given a set of nodes."""
    node_dict = {
        node.node_id: node.get_content(metadata_mode=MetadataMode.NONE)
        for node in nodes
    }

    queries = {}
    relevant_docs = {}
    semaphore = asyncio.Semaphore(max_concurrent)

    async def process_node(node_id: str, text: str):
        async with semaphore:
            query = qa_generate_prompt_tmpl.format(
                context_str=text, num_questions_per_chunk=num_questions_per_chunk
            )
            response = await llm.acomplete(
                query
            )  # Assuming the LLM has an async method

            result = str(response).strip().split("\n")
            questions = [
                re.sub(r"^\d+[\).\s]", "", question).strip() for question in result
            ]
            questions = [question for question in questions if len(question) > 0][
                :num_questions_per_chunk
            ]

            num_questions_generated = len(questions)
            if num_questions_generated < num_questions_per_chunk:
                warnings.warn(
                    f"Fewer questions generated ({num_questions_generated}) "
                    f"than requested ({num_questions_per_chunk})."
                )

            for question in questions:
                question_id = str(uuid.uuid4())
                queries[question_id] = question
                relevant_docs[question_id] = [node_id]

            await asyncio.sleep(delay)

    # Use asyncio.gather to process nodes concurrently
    await async_tqdm.gather(
        *[process_node(node_id, text) for node_id, text in node_dict.items()]
    )

    # construct dataset
    return EmbeddingQAFinetuneDataset(
        queries=queries, corpus=node_dict, relevant_docs=relevant_docs
    )

In [ ]:
import pickle


async def generate_questions(path, source_name):
    nodes = []
    with open(path, "rb") as f:
        document_dict = pickle.load(f)
        for doc_id in document_dict.keys():
            doc: Document = document_dict[doc_id]
            if doc.metadata["tokens"] >= 100_000:
                print("skipping", doc.metadata["tokens"])
                continue
            node = TextNode(text=doc.text, metadata=doc.metadata, id_=doc_id)
            nodes.append(node)

    rag_eval_dataset: EmbeddingQAFinetuneDataset = await generate_qa_embedding_pairs(
        nodes,
        llm=llm,
        num_questions_per_chunk=1,
        qa_generate_prompt_tmpl=STUDENT_AI_QUESTIONS_GENERATE_PROMPT_TMPL,
        max_concurrent=20,  # Adjust this to control concurrency
        delay=0.5,  # Adjust this to add delay between API calls
    )
    rag_eval_dataset.save_json(f"./rag_eval_{source_name}.json")

In [ ]:
# await generate_questions(
#     "../data/chroma-db-langchain/document_dict_langchain.pkl", "langchain"
# )
# await generate_questions(
#     "../data/chroma-db-llama_index/document_dict_llama_index.pkl", "llama_index"
# )
# await generate_questions(
#     "../data/chroma-db-openai_cookbooks/document_dict_openai_cookbooks.pkl",
#     "openai_cookbooks",
# )
# await generate_questions("../data/chroma-db-peft/document_dict_peft.pkl", "peft")
# await generate_questions("../data/chroma-db-trl/document_dict_trl.pkl", "trl")

await generate_questions(
    "../data/chroma-db-tai_blog/document_dict_tai_blog.pkl", "tai_blog"
)

In [ ]:
# # We can also load the dataset from a previously saved json file.
# from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

# rag_eval_dataset = EmbeddingQAFinetuneDataset.from_json("./rag_eval_transformers.json")